In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('../data/vis/author_bg/author2.csv')

In [12]:
df.groupby('subreddit')['id'].count().sort_values(ascending=False)[:10]

subreddit
coys               650
soccer             458
FIFA               241
FantasyPL          130
2007scape           97
ASX_Bets            28
leagueoflegends     27
wow                 27
movies              24
FargoTV             18
Name: id, dtype: int64

In [14]:
df.shape[0]-(650+458+241+130+97+28+27)

378

In [13]:
df.shape[0]

2009

In [2]:
import praw
import pandas as pd
import numpy as np
import requests
from praw.models import MoreComments
import datetime as dt
from datetime import datetime,timedelta
from psaw import PushshiftAPI
from tqdm import tqdm
import pickle
import dill
import tqdm.notebook as tq
from utils import crawler_utils
from utils.crawler_utils import *

In [3]:
headers = api_initialization()
res = requests.get('https://oauth.reddit.com/r/coys/comments/njbsnp/postmatch_thread_leicester_city_vs_spurs_23_may/', headers=headers)

In [ ]:
res.json()[1]

In [4]:
comment_list = res.json()[1]

In [6]:
comment_list

{'kind': 'Listing',
 'data': {'after': None,
  'dist': None,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't1',
    'data': {'subreddit_id': 't5_2ryq7',
     'approved_at_utc': None,
     'author_is_blocked': False,
     'comment_type': None,
     'awarders': [],
     'mod_reason_by': None,
     'banned_by': None,
     'author_flair_type': 'text',
     'total_awards_received': 36,
     'subreddit': 'coys',
     'author_flair_template_id': None,
     'likes': None,
     'replies': {'kind': 'Listing',
      'data': {'after': None,
       'dist': None,
       'modhash': None,
       'geo_filter': '',
       'children': [{'kind': 't1',
         'data': {'subreddit_id': 't5_2ryq7',
          'approved_at_utc': None,
          'author_is_blocked': False,
          'comment_type': None,
          'awarders': [],
          'mod_reason_by': None,
          'banned_by': None,
          'author_flair_type': 'richtext',
          'total_awards_received': 2,
          'subreddit':

In [5]:
def get_comment(comment_list,df,parent,layer):
    comment_list = comment_list['data']['children']
    for comment in comment_list:
        c_text, c_id, c_post, c_subreddit, c_time, c_author, c_ups = None, None, None, None, None, None, None
        if 'body' in comment['data']:
            c_text = comment['data']['body']
        if 'id' in comment['data']:
            c_id = comment['data']['id']
        if 'subreddit' in comment['data']:
            c_subreddit = comment['data']['subreddit']
        if 'created_utc' in comment['data']:
            c_time = datetime.fromtimestamp(int(comment['data']['created_utc']))
        if 'author' in comment['data']:
            c_author = comment['data']['author']
        if 'ups' in comment['data']:
            c_ups = comment['data']['ups']
        if 'replies' not in comment['data']:
            df = df.append(
                {'text': c_text, 'id': c_id, 'subreddit': c_subreddit,
                 'time': c_time, 'author': c_author, 'ups': c_ups, 'parent':parent, 'layer':layer, 'replies':0},
                ignore_index=True)
        if 'replies' in comment['data'] and comment['data']['replies'] != '':
            df = df.append(
                {'text': c_text, 'id': c_id, 'subreddit': c_subreddit,
                 'time': c_time, 'author': c_author, 'ups': c_ups, 'parent':parent, 'layer':layer, 'replies':1},
                ignore_index=True)
            df = get_comment(comment['data']['replies'],df,c_id,layer+1)
    return df

In [6]:
columns = ['text','id','subreddit','time','author','ups','layer','parent']
df=pd.DataFrame(columns=columns)

df = get_comment(comment_list,df,-1,0)

In [14]:
df.loc[df.id=='gz6fq9a']

,text,subreddit,time,author,ups,layer,parent,replies,id


In [ ]:
df.set_index('id')

In [ ]:
df.loc[df.id=='gz6fqhg']

In [7]:
df = df.dropna()

In [8]:
df = df.sort_values(by="ups",ascending=False)
df

,text,id,subreddit,time,author,ups,layer,parent,replies
0,If that was Harry’s last shift for us which is...,gz6fhva,coys,2021-05-24 00:55:09,matip8,1477,0,-1,1.0
8,"My god, we are absolute dog shite, and we've f...",gz6fhbn,coys,2021-05-24 00:55:02,Erlendsaurus,769,0,-1,1.0
17,FUCK ARSENAL. FUCK THE DIVING CUNTS LEICESTER....,gz6fhzx,coys,2021-05-24 00:55:10,psychologicalunit10,466,0,-1,1.0
22,"Gareth Bale, Tottenham\nLegend",gz6fjgq,coys,2021-05-24 00:55:27,Fannypalace,432,0,-1,1.0
33,Imagine bottling it **AGAINST** Tottenham lmao...,gz6fkkb,coys,2021-05-24 00:55:40,byfuryattheheart,349,0,-1,1.0
...,...,...,...,...,...,...,...,...,...
144,Bale knows what he is doing but he cannot say ...,gz6i9i2,coys,2021-05-24 01:16:04,KetoMeUK,6,0,-1,1.0
130,Dele Kane Son with a long embrace 🥺,gz6fwbp,coys,2021-05-24 00:58:02,Mr-Rocafella,6,0,-1,1.0
129,Brendan the bottler,gz6fl0t,coys,2021-05-24 00:55:45,textosterone123,6,0,-1,1.0
142,They gave it as OG for some reason.,gz6j9ff,coys,2021-05-24 01:23:44,[deleted],5,1,gz6ixwn,1.0


In [ ]:
df = df.loc[(df['layer']!=0) | (df['replies']!=0)]

In [50]:
df[:50]

,text,id,subreddit,time,author,ups,layer,parent,replies
0,If that was Harry’s last shift for us which is...,gz6fhva,coys,2021-05-24 00:55:09,matip8,1481,0,-1,1.0
8,"My god, we are absolute dog shite, and we've f...",gz6fhbn,coys,2021-05-24 00:55:02,Erlendsaurus,762,0,-1,1.0
17,FUCK ARSENAL. FUCK THE DIVING CUNTS LEICESTER....,gz6fhzx,coys,2021-05-24 00:55:10,psychologicalunit10,468,0,-1,1.0
22,"Gareth Bale, Tottenham\nLegend",gz6fjgq,coys,2021-05-24 00:55:27,Fannypalace,432,0,-1,1.0
33,Imagine bottling it **AGAINST** Tottenham lmao...,gz6fkkb,coys,2021-05-24 00:55:40,byfuryattheheart,346,0,-1,1.0
9,"“Fuck off, see you in August” is definitely my...",gz6fle7,coys,2021-05-24 00:55:50,aussie_spastic,314,1,gz6fhbn,1.0
39,Fuck you Leicester and fuck you Arsenal,gz6fih9,coys,2021-05-24 00:55:16,aussie_spastic,284,0,-1,1.0
46,Bale scores two\n\nKane wins golden boot and p...,gz6fk6d,coys,2021-05-24 00:55:35,anandgoyal,227,0,-1,1.0
52,How. How did we pull that off.,gz6fiig,coys,2021-05-24 00:55:16,Shweezy,215,0,-1,1.0
63,Fuck off Vardy lol\n\nGet the smug off your fa...,gz6fi93,coys,2021-05-24 00:55:13,u8myramen_y,192,0,-1,1.0


In [44]:
df = df.iloc[:100]

In [51]:
df = df.drop(107)

In [46]:
df

,text,id,subreddit,time,author,ups,layer,parent


In [52]:
s = ''
for i in range(df.shape[0]):
    comment = df.iloc[i]
    if comment['layer']==0:
        s = s+str(comment['id'])+ "[fillcolor=\"#3483ca\" fontcolor=\"#f8f1e0\"]\n"
    else:
        s = s+str(comment['id'])+" -> "+str(comment['parent'])+"\n"
    st = comment['text']
    i=40
    while i < len(st):
        while (st[i]!=' '):
            if i+1<len(st):
                i=i+1
            else:
                break
        st = st[:i]+"<BR/>"+st[i:]
        i = i+40
    s = s+str(comment['id'])+ "[label=<<TABLE ALIGN=\"LEFT\" BORDER=\"0\"><TR><TD BALIGN=\"LEFT\"><FONT>&gt;&gt;"+str(comment['author'])+"<BR/>"+str(st.replace("\n","<BR/>"))+"<BR/></FONT></TD></TR></TABLE>>]\n"

In [53]:
print(s)

gz6fhva[fillcolor="#3483ca" fontcolor="#f8f1e0"]
gz6fhva[label=<<TABLE ALIGN="LEFT" BORDER="0"><TR><TD BALIGN="LEFT"><FONT>&gt;&gt;matip8<BR/>If that was Harry’s last shift for us which<BR/> is likely, he gave a great performance<BR/> and left it all on the pitch, well<BR/> done.    <BR/>     <BR/>Kane was decent to<BR/>o<BR/></FONT></TD></TR></TABLE>>]
gz6fhbn[fillcolor="#3483ca" fontcolor="#f8f1e0"]
gz6fhbn[label=<<TABLE ALIGN="LEFT" BORDER="0"><TR><TD BALIGN="LEFT"><FONT>&gt;&gt;Erlendsaurus<BR/>My god, we are absolute dog shite, and we've<BR/> finished 7th which is an absolute embarrassment.<BR/><BR/>But<BR/> taking 4th away from Leicester and<BR/> giving them nothing from this match<BR/> after Vardy dived like a fucking Stuka<BR/> on that 2nd pen puts such a big smile<BR/> on my face. <BR/><BR/>Fuck off, I'll see you<BR/> in August<BR/></FONT></TD></TR></TABLE>>]
gz6fhzx[fillcolor="#3483ca" fontcolor="#f8f1e0"]
gz6fhzx[label=<<TABLE ALIGN="LEFT" BORDER="0"><TR><TD BALIGN="LEFT"><F

In [ ]:
df.loc[154]['text']

In [ ]:
comment1 = pd.read_csv('../data/comments/author_bg/sentiment/TOT_neg_author_bg_sentiment.csv')
comment2 = pd.read_csv('../data/comments/author_bg/sentiment/TOT_pos_author_bg_sentiment.csv')
comment = pd.concat([comment1,comment2])

In [ ]:
comment.subreddit.unique()

In [ ]:
user = comment.groupby('author').sentiment_score.agg(['mean', 'count'])

In [ ]:
user.describe()

In [ ]:
randomX = np.random.normal(4000,6000,5000)

In [ ]:
listX = []
for i in range(len(randomX)):
    if randomX[i]>0:
        listX.append(int(randomX[i]))

In [ ]:
len(listX)

In [ ]:
print(listX)

In [ ]:
randomY = np.random.normal(0.14,0.17,3754)

In [ ]:
len(randomY)

In [ ]:
randomY.min()

In [ ]:
print(list(randomY))

In [ ]:
s = ''
for i in range(len(randomY)):
    s = s+',{x:'+str(listX[i])+',y:'+str(randomY[i])+'}'

In [ ]:
print(s)

In [1]:
df

NameError: name 'df' is not defined

In [9]:
df.to_csv('../data/sample_comments.csv',index=False)